<a href="https://colab.research.google.com/github/zhamba1130/choshui_groundwater_model/blob/master/pumpingAllocation_choshui2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import gym
from gym import error, spaces, utils
from gym.utils import seeding
import numpy as np  # for array stuff and random
import random
import flopy
import flopy.utils.binaryfile as bf
from skimage.draw import polygon
import os
import shutil


class pumpingAllocationEnvCS2(gym.Env):
    def __init__(self):
        self.seed()
        self.observation_space = spaces.Box(0, np.inf, shape=(6,), dtype=np.float32)
        self.action_space = spaces.MultiDiscrete([11,11])
        
        self.modelname = 'choshui'
        self.mf = flopy.modflow.Modflow.load('choshui.nam', verbose=False, check=False)
        
        #stress period:[ layer, row, column, Q(m^3/hour)]
        self.wel_list = [[0, 318, 190, 0], [0, 317, 190, 0], [0, 316, 190, 0], [0, 315, 189, 0], [0, 315, 190, 0], [0, 315, 191, 0], [0, 314, 189, 0], [0, 314, 190, 0], [0, 314, 191, 0], [0, 313, 189, 0], [0, 313, 190, 0], [0, 313, 191, 0], [0, 313, 192, 0], [0, 312, 189, 0], [0, 312, 190, 0], [0, 312, 191, 0], [0, 312, 192, 0], [0, 311, 188, 0], [0, 311, 189, 0], [0, 311, 190, 0], [0, 311, 191, 0], [0, 311, 192, 0], [0, 311, 193, 0], [0, 310, 189, 0], [0, 310, 190, 0], [0, 310, 191, 0], [0, 310, 192, 0], [0, 310, 193, 0], [0, 309, 189, 0], [0, 309, 190, 0], [0, 309, 191, 0], [0, 309, 192, 0], [0, 309, 193, 0], [0, 309, 194, 0], [0, 308, 189, 0], [0, 308, 190, 0], [0, 308, 191, 0], [0, 308, 192, 0], [0, 308, 193, 0], [0, 308, 194, 0], [0, 308, 195, 0], [0, 307, 189, 0], [0, 307, 190, 0], [0, 307, 191, 0], [0, 307, 192, 0], [0, 307, 193, 0], [0, 307, 194, 0], [0, 307, 195, 0], [0, 307, 196, 0], [0, 306, 189, 0], [0, 306, 190, 0], [0, 306, 191, 0], [0, 306, 192, 0], [0, 306, 193, 0], [0, 306, 194, 0], [0, 306, 195, 0], [0, 306, 196, 0], [0, 305, 189, 0], [0, 305, 190, 0], [0, 305, 191, 0], [0, 305, 192, 0], [0, 305, 193, 0], [0, 305, 194, 0], [0, 305, 195, 0], [0, 305, 196, 0], [0, 305, 197, 0], [0, 304, 189, 0], [0, 304, 190, 0], [0, 304, 191, 0], [0, 304, 192, 0], [0, 304, 193, 0], [0, 304, 194, 0], [0, 304, 195, 0], [0, 304, 196, 0], [0, 304, 197, 0], [0, 304, 198, 0], [0, 303, 190, 0], [0, 303, 191, 0], [0, 303, 192, 0], [0, 303, 193, 0], [0, 303, 194, 0], [0, 303, 195, 0], [0, 303, 196, 0], [0, 303, 197, 0], [0, 303, 198, 0], [0, 303, 199, 0], [0, 302, 190, 0], [0, 302, 191, 0], [0, 302, 192, 0], [0, 302, 193, 0], [0, 302, 194, 0], [0, 302, 195, 0], [0, 302, 196, 0], [0, 302, 197, 0], [0, 302, 198, 0], [0, 302, 199, 0], [0, 302, 200, 0], [0, 301, 190, 0], [0, 301, 191, 0], [0, 301, 192, 0], [0, 301, 193, 0], [0, 301, 194, 0], [0, 301, 195, 0], [0, 301, 196, 0], [0, 301, 197, 0], [0, 301, 198, 0], [0, 301, 199, 0], [0, 301, 200, 0], [0, 300, 190, 0], [0, 300, 191, 0], [0, 300, 192, 0], [0, 300, 193, 0], [0, 300, 194, 0], [0, 300, 195, 0], [0, 300, 196, 0], [0, 300, 197, 0], [0, 300, 198, 0], [0, 300, 199, 0], [0, 300, 200, 0], [0, 299, 190, 0], [0, 299, 191, 0], [0, 299, 192, 0], [0, 299, 193, 0], [0, 299, 194, 0], [0, 299, 195, 0], [0, 299, 196, 0], [0, 299, 197, 0], [0, 299, 198, 0], [0, 299, 199, 0], [0, 299, 200, 0], [0, 299, 201, 0], [0, 298, 190, 0], [0, 298, 191, 0], [0, 298, 192, 0], [0, 298, 193, 0], [0, 298, 194, 0], [0, 298, 195, 0], [0, 298, 196, 0], [0, 298, 197, 0], [0, 298, 198, 0], [0, 298, 199, 0], [0, 298, 200, 0], [0, 298, 201, 0], [0, 297, 190, 0], [0, 297, 191, 0], [0, 297, 192, 0], [0, 297, 193, 0], [0, 297, 194, 0], [0, 297, 195, 0], [0, 297, 196, 0], [0, 297, 197, 0], [0, 297, 198, 0], [0, 297, 199, 0], [0, 297, 200, 0], [0, 296, 191, 0], [0, 296, 192, 0], [0, 296, 193, 0], [0, 296, 194, 0], [0, 296, 195, 0], [0, 296, 196, 0], [0, 296, 197, 0], [0, 296, 198, 0], [0, 296, 199, 0], [0, 296, 200, 0], [0, 295, 191, 0], [0, 295, 192, 0], [0, 295, 193, 0], [0, 295, 194, 0], [0, 295, 195, 0], [0, 295, 196, 0], [0, 295, 197, 0], [0, 295, 198, 0], [0, 295, 199, 0], [0, 295, 200, 0], [0, 294, 192, 0], [0, 294, 193, 0], [0, 294, 194, 0], [0, 294, 195, 0], [0, 294, 196, 0], [0, 294, 197, 0], [0, 294, 198, 0], [0, 294, 199, 0], [0, 294, 200, 0], [0, 293, 192, 0], [0, 293, 193, 0], [0, 293, 194, 0], [0, 293, 195, 0], [0, 293, 196, 0], [0, 293, 197, 0], [0, 293, 198, 0], [0, 293, 199, 0], [0, 293, 200, 0], [0, 292, 193, 0], [0, 292, 194, 0], [0, 292, 195, 0], [0, 292, 196, 0], [0, 292, 197, 0], [0, 292, 198, 0], [0, 292, 199, 0], [0, 291, 193, 0], [0, 291, 194, 0], [0, 291, 195, 0], [0, 291, 196, 0], [0, 291, 197, 0], [0, 291, 198, 0], [0, 291, 199, 0], [0, 290, 193, 0], [0, 290, 194, 0], [0, 290, 195, 0], [0, 290, 196, 0], [0, 290, 197, 0], [0, 290, 198, 0], [0, 290, 199, 0], [0, 289, 194, 0], [0, 289, 195, 0], [0, 289, 196, 0], [0, 289, 197, 0], [0, 289, 198, 0], [0, 289, 199, 0], [0, 288, 194, 0], [0, 288, 195, 0], [0, 288, 196, 0], [0, 288, 197, 0], [0, 288, 198, 0], [0, 288, 199, 0], [0, 288, 200, 0], [0, 287, 194, 0], [0, 287, 195, 0], [0, 287, 196, 0], [0, 287, 197, 0], [0, 287, 198, 0], [0, 287, 199, 0], [0, 287, 200, 0], [0, 286, 195, 0], [0, 286, 196, 0], [0, 286, 197, 0], [0, 286, 198, 0], [0, 286, 199, 0], [0, 286, 200, 0], [0, 285, 195, 0], [0, 285, 196, 0], [0, 285, 197, 0], [0, 285, 198, 0], [0, 285, 199, 0], [0, 285, 200, 0], [0, 285, 201, 0], [0, 284, 195, 0], [0, 284, 196, 0], [0, 284, 197, 0], [0, 284, 198, 0], [0, 284, 199, 0], [0, 284, 200, 0], [0, 284, 201, 0], [0, 283, 194, 0], [0, 283, 195, 0], [0, 283, 196, 0], [0, 283, 197, 0], [0, 283, 198, 0], [0, 283, 199, 0], [0, 283, 200, 0], [0, 283, 201, 0], [0, 282, 192, 0], [0, 282, 193, 0], [0, 282, 194, 0], [0, 282, 195, 0], [0, 282, 196, 0], [0, 282, 197, 0], [0, 282, 198, 0], [0, 282, 199, 0], [0, 282, 200, 0], [0, 282, 201, 0], [0, 282, 202, 0], [0, 281, 190, 0], [0, 281, 191, 0], [0, 281, 192, 0], [0, 281, 193, 0], [0, 281, 194, 0], [0, 281, 195, 0], [0, 281, 196, 0], [0, 281, 197, 0], [0, 281, 198, 0], [0, 281, 199, 0], [0, 281, 200, 0], [0, 281, 201, 0], [0, 281, 202, 0], [0, 280, 188, 0], [0, 280, 189, 0], [0, 280, 190, 0], [0, 280, 191, 0], [0, 280, 192, 0], [0, 280, 193, 0], [0, 280, 194, 0], [0, 280, 195, 0], [0, 280, 196, 0], [0, 280, 197, 0], [0, 280, 198, 0], [0, 280, 199, 0], [0, 280, 200, 0], [0, 280, 201, 0], [0, 280, 202, 0], [0, 280, 203, 0], [0, 279, 185, 0], [0, 279, 186, 0], [0, 279, 187, 0], [0, 279, 188, 0], [0, 279, 189, 0], [0, 279, 190, 0], [0, 279, 191, 0], [0, 279, 192, 0], [0, 279, 193, 0], [0, 279, 194, 0], [0, 279, 195, 0], [0, 279, 196, 0], [0, 279, 197, 0], [0, 279, 198, 0], [0, 279, 199, 0], [0, 279, 200, 0], [0, 279, 201, 0], [0, 279, 202, 0], [0, 279, 203, 0], [0, 278, 182, 0], [0, 278, 183, 0], [0, 278, 184, 0], [0, 278, 185, 0], [0, 278, 186, 0], [0, 278, 187, 0], [0, 278, 188, 0], [0, 278, 189, 0], [0, 278, 190, 0], [0, 278, 191, 0], [0, 278, 192, 0], [0, 278, 193, 0], [0, 278, 194, 0], [0, 278, 195, 0], [0, 278, 196, 0], [0, 278, 197, 0], [0, 278, 198, 0], [0, 278, 199, 0], [0, 278, 200, 0], [0, 278, 201, 0], [0, 278, 202, 0], [0, 278, 203, 0], [0, 277, 178, 0], [0, 277, 179, 0], [0, 277, 180, 0], [0, 277, 181, 0], [0, 277, 182, 0], [0, 277, 183, 0], [0, 277, 184, 0], [0, 277, 185, 0], [0, 277, 186, 0], [0, 277, 187, 0], [0, 277, 188, 0], [0, 277, 189, 0], [0, 277, 190, 0], [0, 277, 191, 0], [0, 277, 192, 0], [0, 277, 193, 0], [0, 277, 194, 0], [0, 277, 195, 0], [0, 277, 196, 0], [0, 277, 197, 0], [0, 277, 198, 0], [0, 277, 199, 0], [0, 277, 200, 0], [0, 277, 201, 0], [0, 277, 202, 0], [0, 277, 203, 0], [0, 276, 177, 0], [0, 276, 178, 0], [0, 276, 179, 0], [0, 276, 180, 0], [0, 276, 181, 0], [0, 276, 182, 0], [0, 276, 183, 0], [0, 276, 184, 0], [0, 276, 185, 0], [0, 276, 186, 0], [0, 276, 187, 0], [0, 276, 188, 0], [0, 276, 189, 0], [0, 276, 190, 0], [0, 276, 191, 0], [0, 276, 192, 0], [0, 276, 193, 0], [0, 276, 194, 0], [0, 276, 195, 0], [0, 276, 196, 0], [0, 276, 197, 0], [0, 276, 198, 0], [0, 276, 199, 0], [0, 276, 200, 0], [0, 276, 201, 0], [0, 276, 202, 0], [0, 276, 203, 0], [0, 276, 204, 0], [0, 275, 176, 0], [0, 275, 177, 0], [0, 275, 178, 0], [0, 275, 179, 0], [0, 275, 180, 0], [0, 275, 181, 0], [0, 275, 182, 0], [0, 275, 183, 0], [0, 275, 184, 0], [0, 275, 185, 0], [0, 275, 186, 0], [0, 275, 187, 0], [0, 275, 188, 0], [0, 275, 189, 0], [0, 275, 190, 0], [0, 275, 191, 0], [0, 275, 192, 0], [0, 275, 193, 0], [0, 275, 194, 0], [0, 275, 195, 0], [0, 275, 196, 0], [0, 275, 197, 0], [0, 275, 198, 0], [0, 275, 199, 0], [0, 275, 200, 0], [0, 275, 201, 0], [0, 275, 202, 0], [0, 275, 203, 0], [0, 275, 204, 0], [0, 274, 175, 0], [0, 274, 176, 0], [0, 274, 177, 0], [0, 274, 178, 0], [0, 274, 179, 0], [0, 274, 180, 0], [0, 274, 181, 0], [0, 274, 182, 0], [0, 274, 183, 0], [0, 274, 184, 0], [0, 274, 185, 0], [0, 274, 186, 0], [0, 274, 187, 0], [0, 274, 188, 0], [0, 274, 189, 0], [0, 274, 190, 0], [0, 274, 191, 0], [0, 274, 192, 0], [0, 274, 193, 0], [0, 274, 194, 0], [0, 274, 195, 0], [0, 274, 196, 0], [0, 274, 197, 0], [0, 274, 198, 0], [0, 274, 199, 0], [0, 274, 200, 0], [0, 274, 201, 0], [0, 274, 202, 0], [0, 274, 203, 0], [0, 274, 204, 0], [0, 274, 205, 0], [0, 273, 175, 0], [0, 273, 176, 0], [0, 273, 177, 0], [0, 273, 178, 0], [0, 273, 179, 0], [0, 273, 180, 0], [0, 273, 181, 0], [0, 273, 182, 0], [0, 273, 183, 0], [0, 273, 184, 0], [0, 273, 185, 0], [0, 273, 186, 0], [0, 273, 187, 0], [0, 273, 188, 0], [0, 273, 189, 0], [0, 273, 190, 0], [0, 273, 191, 0], [0, 273, 192, 0], [0, 273, 193, 0], [0, 273, 194, 0], [0, 273, 195, 0], [0, 273, 196, 0], [0, 273, 197, 0], [0, 273, 198, 0], [0, 273, 199, 0], [0, 273, 200, 0], [0, 273, 201, 0], [0, 273, 202, 0], [0, 273, 203, 0], [0, 273, 204, 0], [0, 273, 205, 0], [0, 272, 174, 0], [0, 272, 175, 0], [0, 272, 176, 0], [0, 272, 177, 0], [0, 272, 178, 0], [0, 272, 179, 0], [0, 272, 180, 0], [0, 272, 181, 0], [0, 272, 182, 0], [0, 272, 183, 0], [0, 272, 184, 0], [0, 272, 185, 0], [0, 272, 186, 0], [0, 272, 187, 0], [0, 272, 188, 0], [0, 272, 189, 0], [0, 272, 190, 0], [0, 272, 191, 0], [0, 272, 192, 0], [0, 272, 193, 0], [0, 272, 194, 0], [0, 272, 195, 0], [0, 272, 196, 0], [0, 272, 197, 0], [0, 272, 198, 0], [0, 272, 199, 0], [0, 272, 200, 0], [0, 272, 201, 0], [0, 272, 202, 0], [0, 272, 203, 0], [0, 272, 204, 0], [0, 272, 205, 0], [0, 272, 206, 0], [0, 271, 174, 0], [0, 271, 175, 0], [0, 271, 176, 0], [0, 271, 177, 0], [0, 271, 178, 0], [0, 271, 179, 0], [0, 271, 180, 0], [0, 271, 181, 0], [0, 271, 182, 0], [0, 271, 183, 0], [0, 271, 184, 0], [0, 271, 185, 0], [0, 271, 186, 0], [0, 271, 187, 0], [0, 271, 188, 0], [0, 271, 189, 0], [0, 271, 190, 0], [0, 271, 191, 0], [0, 271, 192, 0], [0, 271, 193, 0], [0, 271, 194, 0], [0, 271, 195, 0], [0, 271, 196, 0], [0, 271, 197, 0], [0, 271, 198, 0], [0, 271, 199, 0], [0, 271, 200, 0], [0, 271, 201, 0], [0, 271, 202, 0], [0, 271, 203, 0], [0, 271, 204, 0], [0, 271, 205, 0], [0, 271, 206, 0], [0, 271, 207, 0], [0, 270, 173, 0], [0, 270, 174, 0], [0, 270, 175, 0], [0, 270, 176, 0], [0, 270, 177, 0], [0, 270, 178, 0], [0, 270, 179, 0], [0, 270, 180, 0], [0, 270, 181, 0], [0, 270, 182, 0], [0, 270, 183, 0], [0, 270, 184, 0], [0, 270, 185, 0], [0, 270, 186, 0], [0, 270, 187, 0], [0, 270, 188, 0], [0, 270, 189, 0], [0, 270, 190, 0], [0, 270, 191, 0], [0, 270, 192, 0], [0, 270, 193, 0], [0, 270, 194, 0], [0, 270, 195, 0], [0, 270, 196, 0], [0, 270, 197, 0], [0, 270, 198, 0], [0, 270, 199, 0], [0, 270, 200, 0], [0, 270, 201, 0], [0, 270, 202, 0], [0, 270, 203, 0], [0, 270, 204, 0], [0, 270, 205, 0], [0, 270, 206, 0], [0, 270, 207, 0], [0, 270, 208, 0], [0, 269, 173, 0], [0, 269, 174, 0], [0, 269, 175, 0], [0, 269, 176, 0], [0, 269, 177, 0], [0, 269, 178, 0], [0, 269, 179, 0], [0, 269, 180, 0], [0, 269, 181, 0], [0, 269, 182, 0], [0, 269, 183, 0], [0, 269, 184, 0], [0, 269, 185, 0], [0, 269, 186, 0], [0, 269, 187, 0], [0, 269, 188, 0], [0, 269, 189, 0], [0, 269, 190, 0], [0, 269, 191, 0], [0, 269, 192, 0], [0, 269, 193, 0], [0, 269, 194, 0], [0, 269, 195, 0], [0, 269, 196, 0], [0, 269, 197, 0], [0, 269, 198, 0], [0, 269, 199, 0], [0, 269, 200, 0], [0, 269, 201, 0], [0, 269, 202, 0], [0, 269, 203, 0], [0, 269, 204, 0], [0, 269, 205, 0], [0, 269, 206, 0], [0, 269, 207, 0], [0, 269, 208, 0], [0, 269, 209, 0], [0, 268, 172, 0], [0, 268, 173, 0], [0, 268, 174, 0], [0, 268, 175, 0], [0, 268, 176, 0], [0, 268, 177, 0], [0, 268, 178, 0], [0, 268, 179, 0], [0, 268, 180, 0], [0, 268, 181, 0], [0, 268, 182, 0], [0, 268, 183, 0], [0, 268, 184, 0], [0, 268, 185, 0], [0, 268, 186, 0], [0, 268, 187, 0], [0, 268, 188, 0], [0, 268, 189, 0], [0, 268, 190, 0], [0, 268, 191, 0], [0, 268, 192, 0], [0, 268, 193, 0], [0, 268, 194, 0], [0, 268, 195, 0], [0, 268, 196, 0], [0, 268, 197, 0], [0, 268, 198, 0], [0, 268, 199, 0], [0, 268, 200, 0], [0, 268, 201, 0], [0, 268, 202, 0], [0, 268, 203, 0], [0, 268, 204, 0], [0, 268, 205, 0], [0, 268, 206, 0], [0, 268, 207, 0], [0, 268, 208, 0], [0, 268, 209, 0], [0, 268, 210, 0], [0, 267, 172, 0], [0, 267, 173, 0], [0, 267, 174, 0], [0, 267, 175, 0], [0, 267, 176, 0], [0, 267, 177, 0], [0, 267, 178, 0], [0, 267, 179, 0], [0, 267, 180, 0], [0, 267, 181, 0], [0, 267, 182, 0], [0, 267, 183, 0], [0, 267, 184, 0], [0, 267, 185, 0], [0, 267, 186, 0], [0, 267, 187, 0], [0, 267, 188, 0], [0, 267, 189, 0], [0, 267, 190, 0], [0, 267, 191, 0], [0, 267, 192, 0], [0, 267, 193, 0], [0, 267, 194, 0], [0, 267, 195, 0], [0, 267, 196, 0], [0, 267, 197, 0], [0, 267, 198, 0], [0, 267, 199, 0], [0, 267, 200, 0], [0, 267, 201, 0], [0, 267, 202, 0], [0, 267, 203, 0], [0, 267, 204, 0], [0, 267, 205, 0], [0, 267, 206, 0], [0, 267, 207, 0], [0, 267, 208, 0], [0, 267, 209, 0], [0, 267, 210, 0], [0, 267, 211, 0], [0, 267, 212, 0], [0, 266, 172, 0], [0, 266, 173, 0], [0, 266, 174, 0], [0, 266, 175, 0], [0, 266, 176, 0], [0, 266, 177, 0], [0, 266, 178, 0], [0, 266, 179, 0], [0, 266, 180, 0], [0, 266, 181, 0], [0, 266, 182, 0], [0, 266, 183, 0], [0, 266, 184, 0], [0, 266, 185, 0], [0, 266, 186, 0], [0, 266, 187, 0], [0, 266, 188, 0], [0, 266, 189, 0], [0, 266, 190, 0], [0, 266, 191, 0], [0, 266, 192, 0], [0, 266, 193, 0], [0, 266, 194, 0], [0, 266, 195, 0], [0, 266, 196, 0], [0, 266, 197, 0], [0, 266, 198, 0], [0, 266, 199, 0], [0, 266, 200, 0], [0, 266, 201, 0], [0, 266, 202, 0], [0, 266, 203, 0], [0, 266, 204, 0], [0, 266, 205, 0], [0, 266, 206, 0], [0, 266, 207, 0], [0, 266, 208, 0], [0, 266, 209, 0], [0, 266, 210, 0], [0, 266, 211, 0], [0, 266, 212, 0], [0, 266, 213, 0], [0, 265, 171, 0], [0, 265, 172, 0], [0, 265, 173, 0], [0, 265, 174, 0], [0, 265, 175, 0], [0, 265, 176, 0], [0, 265, 177, 0], [0, 265, 178, 0], [0, 265, 179, 0], [0, 265, 180, 0], [0, 265, 181, 0], [0, 265, 182, 0], [0, 265, 183, 0], [0, 265, 184, 0], [0, 265, 185, 0], [0, 265, 186, 0], [0, 265, 187, 0], [0, 265, 188, 0], [0, 265, 189, 0], [0, 265, 190, 0], [0, 265, 191, 0], [0, 265, 192, 0], [0, 265, 193, 0], [0, 265, 194, 0], [0, 265, 195, 0], [0, 265, 196, 0], [0, 265, 197, 0], [0, 265, 198, 0], [0, 265, 199, 0], [0, 265, 200, 0], [0, 265, 201, 0], [0, 265, 202, 0], [0, 265, 203, 0], [0, 265, 204, 0], [0, 265, 205, 0], [0, 265, 206, 0], [0, 265, 207, 0], [0, 265, 208, 0], [0, 265, 209, 0], [0, 265, 210, 0], [0, 265, 211, 0], [0, 265, 212, 0], [0, 265, 213, 0], [0, 265, 214, 0], [0, 265, 215, 0], [0, 264, 171, 0], [0, 264, 172, 0], [0, 264, 173, 0], [0, 264, 174, 0], [0, 264, 175, 0], [0, 264, 176, 0], [0, 264, 177, 0], [0, 264, 178, 0], [0, 264, 179, 0], [0, 264, 180, 0], [0, 264, 181, 0], [0, 264, 182, 0], [0, 264, 183, 0], [0, 264, 184, 0], [0, 264, 185, 0], [0, 264, 186, 0], [0, 264, 187, 0], [0, 264, 188, 0], [0, 264, 189, 0], [0, 264, 190, 0], [0, 264, 191, 0], [0, 264, 192, 0], [0, 264, 193, 0], [0, 264, 194, 0], [0, 264, 195, 0], [0, 264, 196, 0], [0, 264, 197, 0], [0, 264, 198, 0], [0, 264, 199, 0], [0, 264, 200, 0], [0, 264, 201, 0], [0, 264, 202, 0], [0, 264, 203, 0], [0, 264, 204, 0], [0, 264, 205, 0], [0, 264, 206, 0], [0, 264, 207, 0], [0, 264, 208, 0], [0, 264, 209, 0], [0, 264, 210, 0], [0, 264, 211, 0], [0, 264, 212, 0], [0, 264, 213, 0], [0, 264, 214, 0], [0, 264, 215, 0], [0, 263, 170, 0], [0, 263, 171, 0], [0, 263, 172, 0], [0, 263, 173, 0], [0, 263, 174, 0], [0, 263, 175, 0], [0, 263, 176, 0], [0, 263, 177, 0], [0, 263, 178, 0], [0, 263, 179, 0], [0, 263, 180, 0], [0, 263, 181, 0], [0, 263, 182, 0], [0, 263, 183, 0], [0, 263, 184, 0], [0, 263, 185, 0], [0, 263, 186, 0], [0, 263, 187, 0], [0, 263, 188, 0], [0, 263, 189, 0], [0, 263, 190, 0], [0, 263, 191, 0], [0, 263, 192, 0], [0, 263, 193, 0], [0, 263, 194, 0], [0, 263, 195, 0], [0, 263, 196, 0], [0, 263, 197, 0], [0, 263, 198, 0], [0, 263, 199, 0], [0, 263, 200, 0], [0, 263, 201, 0], [0, 263, 202, 0], [0, 263, 203, 0], [0, 263, 204, 0], [0, 263, 205, 0], [0, 263, 206, 0], [0, 263, 207, 0], [0, 263, 208, 0], [0, 263, 209, 0], [0, 263, 210, 0], [0, 263, 211, 0], [0, 263, 212, 0], [0, 263, 213, 0], [0, 263, 214, 0], [0, 263, 215, 0], [0, 262, 170, 0], [0, 262, 171, 0], [0, 262, 172, 0], [0, 262, 173, 0], [0, 262, 174, 0], [0, 262, 175, 0], [0, 262, 176, 0], [0, 262, 177, 0], [0, 262, 178, 0], [0, 262, 179, 0], [0, 262, 180, 0], [0, 262, 181, 0], [0, 262, 182, 0], [0, 262, 183, 0], [0, 262, 184, 0], [0, 262, 185, 0], [0, 262, 186, 0], [0, 262, 187, 0], [0, 262, 188, 0], [0, 262, 189, 0], [0, 262, 190, 0], [0, 262, 191, 0], [0, 262, 192, 0], [0, 262, 193, 0], [0, 262, 194, 0], [0, 262, 195, 0], [0, 262, 196, 0], [0, 262, 197, 0], [0, 262, 198, 0], [0, 262, 199, 0], [0, 262, 200, 0], [0, 262, 201, 0], [0, 262, 202, 0], [0, 262, 203, 0], [0, 262, 204, 0], [0, 262, 205, 0], [0, 262, 206, 0], [0, 262, 207, 0], [0, 262, 208, 0], [0, 262, 209, 0], [0, 262, 210, 0], [0, 262, 211, 0], [0, 262, 212, 0], [0, 262, 213, 0], [0, 262, 214, 0], [0, 262, 215, 0], [0, 261, 170, 0], [0, 261, 171, 0], [0, 261, 172, 0], [0, 261, 173, 0], [0, 261, 174, 0], [0, 261, 175, 0], [0, 261, 176, 0], [0, 261, 177, 0], [0, 261, 178, 0], [0, 261, 179, 0], [0, 261, 180, 0], [0, 261, 181, 0], [0, 261, 182, 0], [0, 261, 183, 0], [0, 261, 184, 0], [0, 261, 185, 0], [0, 261, 186, 0], [0, 261, 187, 0], [0, 261, 188, 0], [0, 261, 189, 0], [0, 261, 190, 0], [0, 261, 191, 0], [0, 261, 192, 0], [0, 261, 193, 0], [0, 261, 194, 0], [0, 261, 195, 0], [0, 261, 196, 0], [0, 261, 197, 0], [0, 261, 198, 0], [0, 261, 199, 0], [0, 261, 200, 0], [0, 261, 201, 0], [0, 261, 202, 0], [0, 261, 203, 0], [0, 261, 204, 0], [0, 261, 205, 0], [0, 261, 206, 0], [0, 261, 207, 0], [0, 261, 208, 0], [0, 261, 209, 0], [0, 261, 210, 0], [0, 261, 211, 0], [0, 261, 212, 0], [0, 261, 213, 0], [0, 261, 214, 0], [0, 261, 215, 0], [0, 260, 169, 0], [0, 260, 170, 0], [0, 260, 171, 0], [0, 260, 172, 0], [0, 260, 173, 0], [0, 260, 174, 0], [0, 260, 175, 0], [0, 260, 176, 0], [0, 260, 177, 0], [0, 260, 178, 0], [0, 260, 179, 0], [0, 260, 180, 0], [0, 260, 181, 0], [0, 260, 182, 0], [0, 260, 183, 0], [0, 260, 184, 0], [0, 260, 185, 0], [0, 260, 186, 0], [0, 260, 187, 0], [0, 260, 188, 0], [0, 260, 189, 0], [0, 260, 190, 0], [0, 260, 191, 0], [0, 260, 192, 0], [0, 260, 193, 0], [0, 260, 194, 0], [0, 260, 195, 0], [0, 260, 196, 0], [0, 260, 197, 0], [0, 260, 198, 0], [0, 260, 199, 0], [0, 260, 200, 0], [0, 260, 201, 0], [0, 260, 202, 0], [0, 260, 203, 0], [0, 260, 204, 0], [0, 260, 205, 0], [0, 260, 206, 0], [0, 260, 207, 0], [0, 260, 208, 0], [0, 260, 209, 0], [0, 260, 210, 0], [0, 260, 211, 0], [0, 260, 212, 0], [0, 260, 213, 0], [0, 260, 214, 0], [0, 260, 215, 0], [0, 259, 169, 0], [0, 259, 170, 0], [0, 259, 171, 0], [0, 259, 172, 0], [0, 259, 173, 0], [0, 259, 174, 0], [0, 259, 175, 0], [0, 259, 176, 0], [0, 259, 177, 0], [0, 259, 178, 0], [0, 259, 179, 0], [0, 259, 180, 0], [0, 259, 181, 0], [0, 259, 182, 0], [0, 259, 183, 0], [0, 259, 184, 0], [0, 259, 185, 0], [0, 259, 186, 0], [0, 259, 187, 0], [0, 259, 188, 0], [0, 259, 189, 0], [0, 259, 190, 0], [0, 259, 191, 0], [0, 259, 192, 0], [0, 259, 193, 0], [0, 259, 194, 0], [0, 259, 195, 0], [0, 259, 196, 0], [0, 259, 197, 0], [0, 259, 198, 0], [0, 259, 199, 0], [0, 259, 200, 0], [0, 259, 201, 0], [0, 259, 202, 0], [0, 259, 203, 0], [0, 259, 204, 0], [0, 259, 205, 0], [0, 259, 206, 0], [0, 259, 207, 0], [0, 259, 208, 0], [0, 259, 209, 0], [0, 259, 210, 0], [0, 259, 211, 0], [0, 259, 212, 0], [0, 259, 213, 0], [0, 259, 214, 0], [0, 259, 215, 0], [0, 258, 168, 0], [0, 258, 169, 0], [0, 258, 170, 0], [0, 258, 171, 0], [0, 258, 172, 0], [0, 258, 173, 0], [0, 258, 174, 0], [0, 258, 175, 0], [0, 258, 176, 0], [0, 258, 177, 0], [0, 258, 178, 0], [0, 258, 179, 0], [0, 258, 180, 0], [0, 258, 181, 0], [0, 258, 182, 0], [0, 258, 183, 0], [0, 258, 184, 0], [0, 258, 185, 0], [0, 258, 186, 0], [0, 258, 187, 0], [0, 258, 188, 0], [0, 258, 189, 0], [0, 258, 190, 0], [0, 258, 191, 0], [0, 258, 192, 0], [0, 258, 193, 0], [0, 258, 194, 0], [0, 258, 195, 0], [0, 258, 196, 0], [0, 258, 197, 0], [0, 258, 198, 0], [0, 258, 199, 0], [0, 258, 200, 0], [0, 258, 201, 0], [0, 258, 202, 0], [0, 258, 203, 0], [0, 258, 204, 0], [0, 258, 205, 0], [0, 258, 206, 0], [0, 258, 207, 0], [0, 258, 208, 0], [0, 258, 209, 0], [0, 258, 210, 0], [0, 258, 211, 0], [0, 258, 212, 0], [0, 258, 213, 0], [0, 258, 214, 0], [0, 258, 215, 0], [0, 257, 168, 0], [0, 257, 169, 0], [0, 257, 170, 0], [0, 257, 171, 0], [0, 257, 172, 0], [0, 257, 173, 0], [0, 257, 174, 0], [0, 257, 175, 0], [0, 257, 176, 0], [0, 257, 177, 0], [0, 257, 178, 0], [0, 257, 179, 0], [0, 257, 180, 0], [0, 257, 181, 0], [0, 257, 182, 0], [0, 257, 183, 0], [0, 257, 184, 0], [0, 257, 185, 0], [0, 257, 186, 0], [0, 257, 187, 0], [0, 257, 188, 0], [0, 257, 189, 0], [0, 257, 190, 0], [0, 257, 191, 0], [0, 257, 192, 0], [0, 257, 193, 0], [0, 257, 194, 0], [0, 257, 195, 0], [0, 257, 196, 0], [0, 257, 197, 0], [0, 257, 198, 0], [0, 257, 199, 0], [0, 257, 200, 0], [0, 257, 201, 0], [0, 257, 202, 0], [0, 257, 203, 0], [0, 257, 204, 0], [0, 257, 205, 0], [0, 257, 206, 0], [0, 257, 207, 0], [0, 257, 208, 0], [0, 257, 209, 0], [0, 257, 210, 0], [0, 257, 211, 0], [0, 257, 212, 0], [0, 257, 213, 0], [0, 257, 214, 0], [0, 257, 215, 0], [0, 256, 168, 0], [0, 256, 169, 0], [0, 256, 170, 0], [0, 256, 171, 0], [0, 256, 172, 0], [0, 256, 173, 0], [0, 256, 174, 0], [0, 256, 175, 0], [0, 256, 176, 0], [0, 256, 177, 0], [0, 256, 178, 0], [0, 256, 179, 0], [0, 256, 180, 0], [0, 256, 181, 0], [0, 256, 182, 0], [0, 256, 183, 0], [0, 256, 184, 0], [0, 256, 185, 0], [0, 256, 186, 0], [0, 256, 187, 0], [0, 256, 188, 0], [0, 256, 189, 0], [0, 256, 190, 0], [0, 256, 191, 0], [0, 256, 192, 0], [0, 256, 193, 0], [0, 256, 194, 0], [0, 256, 195, 0], [0, 256, 196, 0], [0, 256, 197, 0], [0, 256, 198, 0], [0, 256, 199, 0], [0, 256, 200, 0], [0, 256, 201, 0], [0, 256, 202, 0], [0, 256, 203, 0], [0, 256, 204, 0], [0, 256, 205, 0], [0, 256, 206, 0], [0, 256, 207, 0], [0, 256, 208, 0], [0, 256, 209, 0], [0, 256, 210, 0], [0, 256, 211, 0], [0, 256, 212, 0], [0, 256, 213, 0], [0, 256, 214, 0], [0, 256, 215, 0], [0, 255, 168, 0], [0, 255, 169, 0], [0, 255, 170, 0], [0, 255, 171, 0], [0, 255, 172, 0], [0, 255, 173, 0], [0, 255, 174, 0], [0, 255, 175, 0], [0, 255, 176, 0], [0, 255, 177, 0], [0, 255, 178, 0], [0, 255, 179, 0], [0, 255, 180, 0], [0, 255, 181, 0], [0, 255, 182, 0], [0, 255, 183, 0], [0, 255, 184, 0], [0, 255, 185, 0], [0, 255, 186, 0], [0, 255, 187, 0], [0, 255, 188, 0], [0, 255, 189, 0], [0, 255, 190, 0], [0, 255, 191, 0], [0, 255, 192, 0], [0, 255, 193, 0], [0, 255, 194, 0], [0, 255, 195, 0], [0, 255, 196, 0], [0, 255, 197, 0], [0, 255, 198, 0], [0, 255, 199, 0], [0, 255, 200, 0], [0, 255, 201, 0], [0, 255, 202, 0], [0, 255, 203, 0], [0, 255, 204, 0], [0, 255, 205, 0], [0, 255, 206, 0], [0, 255, 207, 0], [0, 255, 208, 0], [0, 255, 209, 0], [0, 255, 210, 0], [0, 255, 211, 0], [0, 255, 212, 0], [0, 255, 213, 0], [0, 255, 214, 0], [0, 255, 215, 0], [0, 254, 167, 0], [0, 254, 168, 0], [0, 254, 169, 0], [0, 254, 170, 0], [0, 254, 171, 0], [0, 254, 172, 0], [0, 254, 173, 0], [0, 254, 174, 0], [0, 254, 175, 0], [0, 254, 176, 0], [0, 254, 177, 0], [0, 254, 178, 0], [0, 254, 179, 0], [0, 254, 180, 0], [0, 254, 181, 0], [0, 254, 182, 0], [0, 254, 183, 0], [0, 254, 184, 0], [0, 254, 185, 0], [0, 254, 186, 0], [0, 254, 187, 0], [0, 254, 188, 0], [0, 254, 189, 0], [0, 254, 190, 0], [0, 254, 191, 0], [0, 254, 192, 0], [0, 254, 193, 0], [0, 254, 194, 0], [0, 254, 195, 0], [0, 254, 196, 0], [0, 254, 197, 0], [0, 254, 198, 0], [0, 254, 199, 0], [0, 254, 200, 0], [0, 254, 201, 0], [0, 254, 202, 0], [0, 254, 203, 0], [0, 254, 204, 0], [0, 254, 205, 0], [0, 254, 206, 0], [0, 254, 207, 0], [0, 254, 208, 0], [0, 254, 209, 0], [0, 254, 210, 0], [0, 254, 211, 0], [0, 254, 212, 0], [0, 254, 213, 0], [0, 254, 214, 0], [0, 254, 215, 0], [0, 253, 167, 0], [0, 253, 168, 0], [0, 253, 169, 0], [0, 253, 170, 0], [0, 253, 171, 0], [0, 253, 172, 0], [0, 253, 173, 0], [0, 253, 174, 0], [0, 253, 175, 0], [0, 253, 176, 0], [0, 253, 177, 0], [0, 253, 178, 0], [0, 253, 179, 0], [0, 253, 180, 0], [0, 253, 181, 0], [0, 253, 182, 0], [0, 253, 183, 0], [0, 253, 184, 0], [0, 253, 185, 0], [0, 253, 186, 0], [0, 253, 187, 0], [0, 253, 188, 0], [0, 253, 189, 0], [0, 253, 190, 0], [0, 253, 191, 0], [0, 253, 192, 0], [0, 253, 193, 0], [0, 253, 194, 0], [0, 253, 195, 0], [0, 253, 196, 0], [0, 253, 197, 0], [0, 253, 198, 0], [0, 253, 199, 0], [0, 253, 200, 0], [0, 253, 201, 0], [0, 253, 202, 0], [0, 253, 203, 0], [0, 253, 204, 0], [0, 253, 205, 0], [0, 253, 206, 0], [0, 253, 207, 0], [0, 253, 208, 0], [0, 253, 209, 0], [0, 253, 210, 0], [0, 253, 211, 0], [0, 253, 212, 0], [0, 253, 213, 0], [0, 253, 214, 0], [0, 253, 215, 0], [0, 252, 167, 0], [0, 252, 168, 0], [0, 252, 169, 0], [0, 252, 170, 0], [0, 252, 171, 0], [0, 252, 172, 0], [0, 252, 173, 0], [0, 252, 174, 0], [0, 252, 175, 0], [0, 252, 176, 0], [0, 252, 177, 0], [0, 252, 178, 0], [0, 252, 179, 0], [0, 252, 180, 0], [0, 252, 181, 0], [0, 252, 182, 0], [0, 252, 183, 0], [0, 252, 184, 0], [0, 252, 185, 0], [0, 252, 186, 0], [0, 252, 187, 0], [0, 252, 188, 0], [0, 252, 189, 0], [0, 252, 190, 0], [0, 252, 191, 0], [0, 252, 192, 0], [0, 252, 193, 0], [0, 252, 194, 0], [0, 252, 195, 0], [0, 252, 196, 0], [0, 252, 197, 0], [0, 252, 198, 0], [0, 252, 199, 0], [0, 252, 200, 0], [0, 252, 201, 0], [0, 252, 202, 0], [0, 252, 203, 0], [0, 252, 204, 0], [0, 252, 205, 0], [0, 252, 206, 0], [0, 252, 207, 0], [0, 252, 208, 0], [0, 252, 209, 0], [0, 252, 210, 0], [0, 252, 211, 0], [0, 252, 212, 0], [0, 252, 213, 0], [0, 252, 214, 0], [0, 252, 215, 0], [0, 251, 166, 0], [0, 251, 167, 0], [0, 251, 168, 0], [0, 251, 169, 0], [0, 251, 170, 0], [0, 251, 171, 0], [0, 251, 172, 0], [0, 251, 173, 0], [0, 251, 174, 0], [0, 251, 175, 0], [0, 251, 176, 0], [0, 251, 177, 0], [0, 251, 178, 0], [0, 251, 179, 0], [0, 251, 180, 0], [0, 251, 181, 0], [0, 251, 182, 0], [0, 251, 183, 0], [0, 251, 184, 0], [0, 251, 185, 0], [0, 251, 186, 0], [0, 251, 187, 0], [0, 251, 188, 0], [0, 251, 189, 0], [0, 251, 190, 0], [0, 251, 191, 0], [0, 251, 192, 0], [0, 251, 193, 0], [0, 251, 194, 0], [0, 251, 195, 0], [0, 251, 196, 0], [0, 251, 197, 0], [0, 251, 198, 0], [0, 251, 199, 0], [0, 251, 200, 0], [0, 251, 201, 0], [0, 251, 202, 0], [0, 251, 203, 0], [0, 251, 204, 0], [0, 251, 205, 0], [0, 251, 206, 0], [0, 251, 207, 0], [0, 251, 208, 0], [0, 251, 209, 0], [0, 251, 210, 0], [0, 251, 211, 0], [0, 251, 212, 0], [0, 251, 213, 0], [0, 250, 167, 0], [0, 250, 168, 0], [0, 250, 169, 0], [0, 250, 170, 0], [0, 250, 171, 0], [0, 250, 172, 0], [0, 250, 173, 0], [0, 250, 174, 0], [0, 250, 175, 0], [0, 250, 176, 0], [0, 250, 177, 0], [0, 250, 178, 0], [0, 250, 179, 0], [0, 250, 180, 0], [0, 250, 181, 0], [0, 250, 182, 0], [0, 250, 183, 0], [0, 250, 184, 0], [0, 250, 185, 0], [0, 250, 186, 0], [0, 250, 187, 0], [0, 250, 188, 0], [0, 250, 189, 0], [0, 250, 190, 0], [0, 250, 191, 0], [0, 250, 192, 0], [0, 250, 193, 0], [0, 250, 194, 0], [0, 250, 195, 0], [0, 250, 196, 0], [0, 250, 197, 0], [0, 250, 198, 0], [0, 250, 199, 0], [0, 250, 200, 0], [0, 250, 201, 0], [0, 250, 202, 0], [0, 250, 203, 0], [0, 250, 204, 0], [0, 250, 205, 0], [0, 250, 206, 0], [0, 250, 207, 0], [0, 250, 208, 0], [0, 250, 209, 0], [0, 250, 210, 0], [0, 250, 211, 0], [0, 250, 212, 0], [0, 249, 167, 0], [0, 249, 168, 0], [0, 249, 169, 0], [0, 249, 170, 0], [0, 249, 171, 0], [0, 249, 172, 0], [0, 249, 173, 0], [0, 249, 174, 0], [0, 249, 175, 0], [0, 249, 176, 0], [0, 249, 177, 0], [0, 249, 178, 0], [0, 249, 179, 0], [0, 249, 180, 0], [0, 249, 181, 0], [0, 249, 182, 0], [0, 249, 183, 0], [0, 249, 184, 0], [0, 249, 185, 0], [0, 249, 186, 0], [0, 249, 187, 0], [0, 249, 188, 0], [0, 249, 189, 0], [0, 249, 190, 0], [0, 249, 191, 0], [0, 249, 192, 0], [0, 249, 193, 0], [0, 249, 194, 0], [0, 249, 195, 0], [0, 249, 196, 0], [0, 249, 197, 0], [0, 249, 198, 0], [0, 249, 199, 0], [0, 249, 200, 0], [0, 249, 201, 0], [0, 249, 202, 0], [0, 249, 203, 0], [0, 249, 204, 0], [0, 249, 205, 0], [0, 249, 206, 0], [0, 249, 207, 0], [0, 249, 208, 0], [0, 249, 209, 0], [0, 249, 210, 0], [0, 248, 167, 0], [0, 248, 168, 0], [0, 248, 169, 0], [0, 248, 170, 0], [0, 248, 171, 0], [0, 248, 172, 0], [0, 248, 173, 0], [0, 248, 174, 0], [0, 248, 175, 0], [0, 248, 176, 0], [0, 248, 177, 0], [0, 248, 178, 0], [0, 248, 179, 0], [0, 248, 180, 0], [0, 248, 181, 0], [0, 248, 182, 0], [0, 248, 183, 0], [0, 248, 184, 0], [0, 248, 185, 0], [0, 248, 186, 0], [0, 248, 187, 0], [0, 248, 188, 0], [0, 248, 189, 0], [0, 248, 190, 0], [0, 248, 191, 0], [0, 248, 192, 0], [0, 248, 193, 0], [0, 248, 194, 0], [0, 248, 195, 0], [0, 248, 196, 0], [0, 248, 197, 0], [0, 248, 198, 0], [0, 248, 199, 0], [0, 248, 200, 0], [0, 248, 201, 0], [0, 248, 202, 0], [0, 248, 203, 0], [0, 248, 204, 0], [0, 248, 205, 0], [0, 248, 206, 0], [0, 248, 207, 0], [0, 248, 208, 0], [0, 248, 209, 0], [0, 247, 168, 0], [0, 247, 169, 0], [0, 247, 170, 0], [0, 247, 171, 0], [0, 247, 172, 0], [0, 247, 173, 0], [0, 247, 174, 0], [0, 247, 175, 0], [0, 247, 176, 0], [0, 247, 177, 0], [0, 247, 178, 0], [0, 247, 179, 0], [0, 247, 180, 0], [0, 247, 181, 0], [0, 247, 182, 0], [0, 247, 183, 0], [0, 247, 184, 0], [0, 247, 185, 0], [0, 247, 186, 0], [0, 247, 187, 0], [0, 247, 188, 0], [0, 247, 189, 0], [0, 247, 190, 0], [0, 247, 191, 0], [0, 247, 192, 0], [0, 247, 193, 0], [0, 247, 194, 0], [0, 247, 195, 0], [0, 247, 196, 0], [0, 247, 197, 0], [0, 247, 198, 0], [0, 247, 199, 0], [0, 247, 200, 0], [0, 247, 201, 0], [0, 247, 202, 0], [0, 247, 203, 0], [0, 247, 204, 0], [0, 247, 205, 0], [0, 247, 206, 0], [0, 247, 207, 0], [0, 247, 208, 0], [0, 246, 168, 0], [0, 246, 169, 0], [0, 246, 170, 0], [0, 246, 171, 0], [0, 246, 172, 0], [0, 246, 173, 0], [0, 246, 174, 0], [0, 246, 175, 0], [0, 246, 176, 0], [0, 246, 177, 0], [0, 246, 178, 0], [0, 246, 179, 0], [0, 246, 180, 0], [0, 246, 181, 0], [0, 246, 182, 0], [0, 246, 183, 0], [0, 246, 184, 0], [0, 246, 185, 0], [0, 246, 186, 0], [0, 246, 187, 0], [0, 246, 188, 0], [0, 246, 189, 0], [0, 246, 190, 0], [0, 246, 191, 0], [0, 246, 192, 0], [0, 246, 193, 0], [0, 246, 194, 0], [0, 246, 195, 0], [0, 246, 196, 0], [0, 246, 197, 0], [0, 246, 198, 0], [0, 246, 199, 0], [0, 246, 200, 0], [0, 246, 201, 0], [0, 246, 202, 0], [0, 246, 203, 0], [0, 246, 204, 0], [0, 246, 205, 0], [0, 246, 206, 0], [0, 246, 207, 0], [0, 245, 168, 0], [0, 245, 169, 0], [0, 245, 170, 0], [0, 245, 171, 0], [0, 245, 172, 0], [0, 245, 173, 0], [0, 245, 174, 0], [0, 245, 175, 0], [0, 245, 176, 0], [0, 245, 177, 0], [0, 245, 178, 0], [0, 245, 179, 0], [0, 245, 180, 0], [0, 245, 181, 0], [0, 245, 182, 0], [0, 245, 183, 0], [0, 245, 184, 0], [0, 245, 185, 0], [0, 245, 186, 0], [0, 245, 187, 0], [0, 245, 188, 0], [0, 245, 189, 0], [0, 245, 190, 0], [0, 245, 191, 0], [0, 245, 192, 0], [0, 245, 193, 0], [0, 245, 194, 0], [0, 245, 195, 0], [0, 245, 196, 0], [0, 245, 197, 0], [0, 245, 198, 0], [0, 245, 199, 0], [0, 245, 200, 0], [0, 245, 201, 0], [0, 245, 202, 0], [0, 245, 203, 0], [0, 245, 204, 0], [0, 245, 205, 0], [0, 245, 206, 0], [0, 244, 168, 0], [0, 244, 169, 0], [0, 244, 170, 0], [0, 244, 171, 0], [0, 244, 172, 0], [0, 244, 173, 0], [0, 244, 174, 0], [0, 244, 175, 0], [0, 244, 176, 0], [0, 244, 177, 0], [0, 244, 178, 0], [0, 244, 179, 0], [0, 244, 180, 0], [0, 244, 181, 0], [0, 244, 182, 0], [0, 244, 183, 0], [0, 244, 184, 0], [0, 244, 185, 0], [0, 244, 186, 0], [0, 244, 187, 0], [0, 244, 188, 0], [0, 244, 189, 0], [0, 244, 190, 0], [0, 244, 191, 0], [0, 244, 192, 0], [0, 244, 193, 0], [0, 244, 194, 0], [0, 244, 195, 0], [0, 244, 196, 0], [0, 244, 197, 0], [0, 244, 198, 0], [0, 244, 199, 0], [0, 244, 200, 0], [0, 244, 201, 0], [0, 244, 202, 0], [0, 244, 203, 0], [0, 244, 204, 0], [0, 244, 205, 0], [0, 243, 169, 0], [0, 243, 170, 0], [0, 243, 171, 0], [0, 243, 172, 0], [0, 243, 173, 0], [0, 243, 174, 0], [0, 243, 175, 0], [0, 243, 176, 0], [0, 243, 177, 0], [0, 243, 178, 0], [0, 243, 179, 0], [0, 243, 180, 0], [0, 243, 181, 0], [0, 243, 182, 0], [0, 243, 183, 0], [0, 243, 184, 0], [0, 243, 185, 0], [0, 243, 186, 0], [0, 243, 187, 0], [0, 243, 188, 0], [0, 243, 189, 0], [0, 243, 190, 0], [0, 243, 191, 0], [0, 243, 192, 0], [0, 243, 193, 0], [0, 243, 194, 0], [0, 243, 195, 0], [0, 243, 196, 0], [0, 243, 197, 0], [0, 243, 198, 0], [0, 243, 199, 0], [0, 243, 200, 0], [0, 243, 201, 0], [0, 243, 202, 0], [0, 243, 203, 0], [0, 243, 204, 0], [0, 242, 169, 0], [0, 242, 170, 0], [0, 242, 171, 0], [0, 242, 172, 0], [0, 242, 173, 0], [0, 242, 174, 0], [0, 242, 175, 0], [0, 242, 176, 0], [0, 242, 177, 0], [0, 242, 178, 0], [0, 242, 179, 0], [0, 242, 180, 0], [0, 242, 181, 0], [0, 242, 182, 0], [0, 242, 183, 0], [0, 242, 184, 0], [0, 242, 185, 0], [0, 242, 186, 0], [0, 242, 187, 0], [0, 242, 188, 0], [0, 242, 189, 0], [0, 242, 190, 0], [0, 242, 191, 0], [0, 242, 192, 0], [0, 242, 193, 0], [0, 242, 194, 0], [0, 242, 195, 0], [0, 242, 196, 0], [0, 242, 197, 0], [0, 242, 198, 0], [0, 242, 199, 0], [0, 242, 200, 0], [0, 242, 201, 0], [0, 242, 202, 0], [0, 242, 203, 0], [0, 241, 170, 0], [0, 241, 171, 0], [0, 241, 172, 0], [0, 241, 173, 0], [0, 241, 174, 0], [0, 241, 175, 0], [0, 241, 176, 0], [0, 241, 177, 0], [0, 241, 178, 0], [0, 241, 179, 0], [0, 241, 180, 0], [0, 241, 181, 0], [0, 241, 182, 0], [0, 241, 183, 0], [0, 241, 184, 0], [0, 241, 185, 0], [0, 241, 186, 0], [0, 241, 187, 0], [0, 241, 188, 0], [0, 241, 189, 0], [0, 241, 190, 0], [0, 241, 191, 0], [0, 241, 192, 0], [0, 241, 193, 0], [0, 241, 194, 0], [0, 241, 195, 0], [0, 241, 196, 0], [0, 241, 197, 0], [0, 241, 198, 0], [0, 241, 199, 0], [0, 241, 200, 0], [0, 241, 201, 0], [0, 241, 202, 0], [0, 240, 170, 0], [0, 240, 171, 0], [0, 240, 172, 0], [0, 240, 173, 0], [0, 240, 174, 0], [0, 240, 175, 0], [0, 240, 176, 0], [0, 240, 177, 0], [0, 240, 178, 0], [0, 240, 179, 0], [0, 240, 180, 0], [0, 240, 181, 0], [0, 240, 182, 0], [0, 240, 183, 0], [0, 240, 184, 0], [0, 240, 185, 0], [0, 240, 186, 0], [0, 240, 187, 0], [0, 240, 188, 0], [0, 240, 189, 0], [0, 240, 190, 0], [0, 240, 191, 0], [0, 240, 192, 0], [0, 240, 193, 0], [0, 240, 194, 0], [0, 240, 195, 0], [0, 240, 196, 0], [0, 240, 197, 0], [0, 240, 198, 0], [0, 240, 199, 0], [0, 240, 200, 0], [0, 240, 201, 0], [0, 240, 202, 0], [0, 239, 170, 0], [0, 239, 171, 0], [0, 239, 172, 0], [0, 239, 173, 0], [0, 239, 174, 0], [0, 239, 175, 0], [0, 239, 176, 0], [0, 239, 177, 0], [0, 239, 178, 0], [0, 239, 179, 0], [0, 239, 180, 0], [0, 239, 181, 0], [0, 239, 182, 0], [0, 239, 183, 0], [0, 239, 184, 0], [0, 239, 185, 0], [0, 239, 186, 0], [0, 239, 187, 0], [0, 239, 188, 0], [0, 239, 189, 0], [0, 239, 190, 0], [0, 239, 191, 0], [0, 239, 192, 0], [0, 239, 193, 0], [0, 239, 194, 0], [0, 239, 195, 0], [0, 239, 196, 0], [0, 239, 197, 0], [0, 239, 198, 0], [0, 239, 199, 0], [0, 239, 200, 0], [0, 239, 201, 0], [0, 239, 202, 0], [0, 238, 171, 0], [0, 238, 172, 0], [0, 238, 173, 0], [0, 238, 174, 0], [0, 238, 175, 0], [0, 238, 176, 0], [0, 238, 177, 0], [0, 238, 178, 0], [0, 238, 179, 0], [0, 238, 180, 0], [0, 238, 181, 0], [0, 238, 182, 0], [0, 238, 183, 0], [0, 238, 184, 0], [0, 238, 185, 0], [0, 238, 186, 0], [0, 238, 187, 0], [0, 238, 188, 0], [0, 238, 189, 0], [0, 238, 190, 0], [0, 238, 191, 0], [0, 238, 192, 0], [0, 238, 193, 0], [0, 238, 194, 0], [0, 238, 195, 0], [0, 238, 196, 0], [0, 238, 197, 0], [0, 238, 198, 0], [0, 238, 199, 0], [0, 238, 200, 0], [0, 238, 201, 0], [0, 238, 202, 0], [0, 237, 171, 0], [0, 237, 172, 0], [0, 237, 173, 0], [0, 237, 174, 0], [0, 237, 175, 0], [0, 237, 176, 0], [0, 237, 177, 0], [0, 237, 178, 0], [0, 237, 179, 0], [0, 237, 180, 0], [0, 237, 181, 0], [0, 237, 182, 0], [0, 237, 183, 0], [0, 237, 184, 0], [0, 237, 185, 0], [0, 237, 186, 0], [0, 237, 187, 0], [0, 237, 188, 0], [0, 237, 189, 0], [0, 237, 190, 0], [0, 237, 191, 0], [0, 237, 192, 0], [0, 237, 193, 0], [0, 237, 194, 0], [0, 237, 195, 0], [0, 237, 196, 0], [0, 237, 197, 0], [0, 237, 198, 0], [0, 237, 199, 0], [0, 237, 200, 0], [0, 237, 201, 0], [0, 237, 202, 0], [0, 236, 172, 0], [0, 236, 173, 0], [0, 236, 174, 0], [0, 236, 175, 0], [0, 236, 176, 0], [0, 236, 177, 0], [0, 236, 178, 0], [0, 236, 179, 0], [0, 236, 180, 0], [0, 236, 181, 0], [0, 236, 182, 0], [0, 236, 183, 0], [0, 236, 184, 0], [0, 236, 185, 0], [0, 236, 186, 0], [0, 236, 187, 0], [0, 236, 188, 0], [0, 236, 189, 0], [0, 236, 190, 0], [0, 236, 191, 0], [0, 236, 192, 0], [0, 236, 193, 0], [0, 236, 194, 0], [0, 236, 195, 0], [0, 236, 196, 0], [0, 236, 197, 0], [0, 236, 198, 0], [0, 236, 199, 0], [0, 236, 200, 0], [0, 236, 201, 0], [0, 236, 202, 0], [0, 235, 172, 0], [0, 235, 173, 0], [0, 235, 174, 0], [0, 235, 175, 0], [0, 235, 176, 0], [0, 235, 177, 0], [0, 235, 178, 0], [0, 235, 179, 0], [0, 235, 180, 0], [0, 235, 181, 0], [0, 235, 182, 0], [0, 235, 183, 0], [0, 235, 184, 0], [0, 235, 185, 0], [0, 235, 186, 0], [0, 235, 187, 0], [0, 235, 188, 0], [0, 235, 189, 0], [0, 235, 190, 0], [0, 235, 191, 0], [0, 235, 192, 0], [0, 235, 193, 0], [0, 235, 194, 0], [0, 235, 195, 0], [0, 235, 196, 0], [0, 235, 197, 0], [0, 235, 198, 0], [0, 235, 199, 0], [0, 235, 200, 0], [0, 235, 201, 0], [0, 235, 202, 0], [0, 234, 172, 0], [0, 234, 173, 0], [0, 234, 174, 0], [0, 234, 175, 0], [0, 234, 176, 0], [0, 234, 177, 0], [0, 234, 178, 0], [0, 234, 179, 0], [0, 234, 180, 0], [0, 234, 181, 0], [0, 234, 182, 0], [0, 234, 183, 0], [0, 234, 184, 0], [0, 234, 185, 0], [0, 234, 186, 0], [0, 234, 187, 0], [0, 234, 188, 0], [0, 234, 189, 0], [0, 234, 190, 0], [0, 234, 191, 0], [0, 234, 192, 0], [0, 234, 193, 0], [0, 234, 194, 0], [0, 234, 195, 0], [0, 234, 196, 0], [0, 234, 197, 0], [0, 234, 198, 0], [0, 234, 199, 0], [0, 234, 200, 0], [0, 234, 201, 0], [0, 234, 202, 0], [0, 233, 176, 0], [0, 233, 177, 0], [0, 233, 178, 0], [0, 233, 179, 0], [0, 233, 180, 0], [0, 233, 181, 0], [0, 233, 182, 0], [0, 233, 183, 0], [0, 233, 184, 0], [0, 233, 185, 0], [0, 233, 186, 0], [0, 233, 187, 0], [0, 233, 188, 0], [0, 233, 189, 0], [0, 233, 190, 0], [0, 233, 191, 0], [0, 233, 192, 0], [0, 233, 193, 0], [0, 233, 194, 0], [0, 233, 195, 0], [0, 233, 196, 0], [0, 233, 197, 0], [0, 233, 198, 0], [0, 233, 199, 0], [0, 233, 200, 0], [0, 233, 201, 0], [0, 233, 202, 0], [0, 232, 180, 0], [0, 232, 181, 0], [0, 232, 182, 0], [0, 232, 183, 0], [0, 232, 184, 0], [0, 232, 185, 0], [0, 232, 186, 0], [0, 232, 187, 0], [0, 232, 188, 0], [0, 232, 189, 0], [0, 232, 190, 0], [0, 232, 191, 0], [0, 232, 192, 0], [0, 232, 193, 0], [0, 232, 194, 0], [0, 232, 195, 0], [0, 232, 196, 0], [0, 232, 197, 0], [0, 232, 198, 0], [0, 232, 199, 0], [0, 232, 200, 0], [0, 232, 201, 0], [0, 232, 202, 0], [0, 232, 203, 0], [0, 231, 190, 0], [0, 231, 191, 0], [0, 231, 192, 0], [0, 231, 193, 0], [0, 231, 194, 0], [0, 231, 195, 0], [0, 231, 196, 0], [0, 231, 197, 0], [0, 231, 198, 0], [0, 231, 199, 0], [0, 231, 200, 0], [0, 231, 201, 0], [0, 231, 202, 0], [0, 231, 203, 0], [0, 230, 198, 0], [0, 230, 199, 0], [0, 230, 200, 0], [0, 230, 201, 0], [0, 230, 202, 0], [0, 230, 203, 0], [0, 229, 200, 0], [0, 229, 201, 0], [0, 229, 202, 0], [0, 229, 203, 0]]
        self.obs_wels = {'Yuanzhang':[82,82], 'Huwei':[109,131], 'Xigang':[175,74], 'Shiliu':[111,195], 
                         'Shinmin':[148,238], 'Tianzhong':[170,195], 'Huatan':[246,177], 'Xihu':[213,151],
                         'Xiluo':[145,145], 'Ganyuan':[156,174]}
        self.obs_wels_h_former = [33.98, 13.39, 19.24, 62.53, 116.57, 39.90, 0.48, 11.50, 22.36, 3.56]
        
        self.max_storage = 650    #最大有效庫容(萬噸)
        self.t_final = 108        #總時間(旬)
        self.ifRandtt = False     #是否隨機起始月份
        self.day = [10, 10, 11, 10, 10, 8, 10, 10, 11, 10, 10, 10, 10, 10, 11, 10, 10, 10, 10, 10, 11, 10, 10, 11, 10, 10, 10, 10, 10, 11, 10, 10, 10, 10, 10, 11]

        #水庫入流(cms)
        inflow2011 = np.array([28.19, 35.16, 31.6, 30.85, 30.82, 31.82, 30.08, 31.45, 35.13, 31.2, 27.36, 30.61,
                               28.95, 70.84, 80.38, 70.25, 69.44, 142.31, 95.82, 202.46, 163.96, 106.54, 119.94, 225.82,
                               143.13, 79.82, 69.27, 184.79, 103.49, 71.56, 62.46, 101.25, 65.71, 55.76, 53.55, 40.02])        
        inflow2012 = np.array([47.01, 52.54, 59.94, 47.17, 41.4, 53.13, 54.73, 52.25, 37.31, 36.2, 52.9, 128.01,
                               233.28, 203.46, 283.31, 222.95, 1991.91, 560.25, 190.15, 119.98, 112.22, 1783.61, 239.12, 346.57,
                               220.86, 112.52, 84.06, 63.38, 61.29, 62.91, 63.42, 59.13, 60.17, 110.04, 78.91, 50.41])
        inflow2013 = np.array([46.62, 42.68, 39.33, 40.2, 38.15, 36.8, 33.75, 30.91, 27.48, 81.08, 94.95, 156.16,
                               214.37, 620.89, 756.25, 209.41, 147.6, 112.83, 128.85, 802.45, 197.72 ,114.72, 103.4, 729.87,
                               381.21, 154.64, 1026.36, 169.86, 96.3, 76.59, 72.87, 60.84, 45.06, 38.07, 49.6, 45.36])
        inflow2014 = np.array([35.16, 29.92, 31.63, 40.14, 39.21, 36.36, 36.21, 47.06, 41.6, 49.84, 37.57, 40.69,
                               104.9, 206.81, 335.87, 339.45, 241.66, 140.8, 113.23, 80.18, 573.24, 150.86, 124.13,
                               95.45, 78.77, 68.35, 93.57, 56.64, 55.47, 52.36, 40.95, 31.62, 28, 20, 21.11, 22.06])
        inflow2015 = np.array([22.73, 24.04, 29.82, 31.08, 30.51, 28.89, 26.93, 23.33, 23.58, 24.2, 24.55, 33.07,
                               33.84, 34.13, 275.15, 136.77, 72.83, 69.92, 62.69, 85.31, 80.24, 150.24, 153.51, 217.92,
                               194.43, 85.39, 327.55, 138.34, 83.23, 61.53, 57.21, 54.92, 30.57, 25.03, 24.18,22.63])
        inflow2016 = np.array([39.25, 55.35, 119.13, 126.52, 70.85, 52.49, 49.45, 245.95, 241.53, 126.02, 190.42, 156.69,
                               108.2, 88.39, 83.48, 108.91, 291.34, 187.39, 425.33, 219.68, 125.71, 100.86, 101.36, 81.15,
                               76.17, 639.94, 600.31, 343.36, 216.32, 172.22, 97.5, 84.51, 61.49, 53.56, 42.56, 41.15])
        inflow2017 = np.array([40.45, 37.55, 37.67, 40.57, 41.71, 41.57, 41.12, 36.54, 35.46, 33.43, 38.83, 142.2,
                               98.84, 99.64, 181.27, 1903.6, 973.34, 339.97, 251.87, 165.1, 435.63, 271.62, 121.52, 130.68,
                               103.8, 77.23, 68.94, 64.43, 309.45, 90.53, 67.1, 64.05, 48.78, 34.69, 37.75, 37.18])        
        inflow2018 = np.array([72.26, 78.69, 51.54, 61.75, 70.23, 52.46, 57.92, 70.66, 44.88, 41.09, 50.57, 52.71,
                               50.46, 57.96, 43.19, 53.65, 83.92, 120.05, 161.98, 196.04, 116.09, 92.52, 142.93, 387.52,
                               188.2, 466.82, 177.59, 90.62, 79.4, 62.59, 58.27, 51.44, 38.01, 30.19, 25.98, 21.73])
        inflow2019 = np.array([27.21, 32.85, 31.77, 33.24, 30.96, 32.91, 36.47, 39.37, 32.78, 35.37, 41.78, 66.32,
                               187.17, 617.27, 429.99, 251.6, 1022.87, 247.64, 152.5, 132.02, 131, 148.47, 453.5, 663.9,
                               242.95, 124.5, 97.81, 80.77, 68.23, 63.13, 47.04, 44.22, 35.91, 28.78, 29.46, 29.47])
        inflow2020 = np.array([27.58, 32.98, 31.53, 31.46, 31.41, 30.7, 29.77, 29.98, 30.69, 31.43, 29.13, 32.84,
                               32.79, 38.86, 253.68, 154.8, 125.59, 64.47, 83.24, 70.37, 81.42, 80.68, 84.59, 67.02,
                               53.97, 54.59, 50.44, 52.26, 54.83, 54.69, 39.56, 29.8, 23.69, 19.71, 17.27, 17.89])
        inflow2021 = np.array([21.41, 26.03, 31.64, 31.55, 32.66, 29.87, 26.04, 22.66, 22.61, 21.25, 20.89, 24.52,
                               24.74, 23.77, 30.82, 100.06, 82.36, 137.77, 96.12, 89.15, 115.15, 659.83, 324, 152.47,
                               109.56, 122.2, 78.02, 61.83, 372.54, 98.62, 68.79, 55.09, 39.36, 37.08, 26.44, 35.66])
        
        self.rivers = [inflow2011, inflow2012, inflow2013, inflow2014, inflow2015, inflow2016, 
                       inflow2017, inflow2018, inflow2019, inflow2020, inflow2021]
        self.river_p = [1/11] * 11      #入流情境機率
        
        #農業水需求(萬噸)
        demandMain = np.array([5151.27,5151.27,5151.27,5499.27,5499.27,5499.27,6272.73,6272.73,6272.73,7290.97,7290.97,7290.97,
                               7416.47,7416.47,7416.47,5630.97,5630.97,5630.97,11506.77,11506.77,11506.77,13312.80,13312.80,13312.80,
                               3236.07,3236.07,3236.07,11759.03,11759.03,11759.03,5001.33,5001.33,5001.33,3749.30,3749.30,3749.30])
        demand110Plan = np.array([4679.42,4679.42,4679.42,4449.60,4449.60,4449.60,5658.34,5658.34,5658.34,5920.13,5920.13,5920.13,
                                  7964.35,7964.35,7964.35,11511.07,11677.82,11911.97,13084.42,12838.18,13756.61,13876.70,13876.70,13876.70,
                                  13802.40,13802.40,13802.40,9688.03,9405.50,7634.65,6241.54,6241.54,6241.54,5791.39,5791.39,5482.94])
        demand110 = np.array([2473.63, 2819.23, 3576.36, 3175.11, 4449.60, 3559.68, 5658.34, 5658.34, 6224.17, 5868.29, 5920.13, 5920.13,
                              7409.66, 7409.6, 8150.63, 3931.20, 5356.80, 4060.80, 6879.95, 8705.66, 9730.39, 3244.49, 8558.09, 9836.64,
                              9072.00, 8251.20, 9072.00, 8408.45, 8408.45, 8398.11, 6241.54, 6241.54, 3382.30, 2766.53, 2832.88, 3433.80])
        demand109 = np.array([2502.75, 4069.53, 5147.37, 4450.46, 4450.46, 3560.37, 5658.34, 5658.34, 6224.17, 5920.13, 5920.13, 5920.13, 
                              7964.35, 7964.35, 5229.29, 8165.84, 8754.57, 8427.28, 9731.23, 9775.30, 11774.51, 10844.93, 10844.93, 11929.42, 
                              10659.17, 10659.17, 10659.17, 8840.45, 8840.45, 7553.87, 6241.54, 5923.84, 3064.61, 2766.53, 2766.53, 3043.18])
        
        self.demands = [demandMain, demand110Plan, demand110, demand109]
        self.demands_p = [1, 0, 0, 0]      #水需求情境機率
        
        
    def pump(self, p, wels):
        ly1, ly2 = [], []
        for i in wels:
            i[3] = p
            temp1, temp2 = i[:], i[:]
            i[3] *= 0.6

            temp1[0] = 1
            temp1[3] *= 0.3
            ly1.append(temp1)
            temp2[0] = 2
            temp2[3] *= 0.1
            ly2.append(temp2)

        return wels + ly1 + ly2  
    
    
    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]
        
        
    def reset(self):
        #============================檔案還原======================================
        copyDir = 'C:/Users/johnny/Desktop/file'# 複製資料夾
        putDir = 'C:/Users/johnny/Desktop/loop test'# 複製要放的資料夾

        for root, dirs, files in os.walk(copyDir):
            # 針對檔案單層複製
            for fileName in files:

                # 取得要複製檔案的完整路徑
                fileP = root + '/' + fileName
                # 取得要複製檔案放置的完整路徑
                newFP = putDir + '/' + fileName

                # 進行複製 (有檔案會直接覆蓋)
                shutil.copyfile(
                    fileP,
                    newFP
                    )
                
        #======================get boundary from BAS file=========================
        path = 'choshui_original.bas'      
        l = 0
        bound, layer, b = [], [], []

        with open(path) as f:
            for line in f.readlines():
                if 'Layer' in line:
                    l += 1

                elif l>0:
                    s = line.split(' ')

                    for j in s:
                        if j != '' :
                            b.append(int(j)) 

                    if len(b) >= 309:
                        layer.append(b)
                        b=[]

                    if len(layer) >= 400:
                        bound.append(layer)
                        layer =[]

                    if len(bound) == 9:
                        break

        self.ibound = np.array(bound)
        
        #==============================Initial Settings===================================
        self.t = 0    #第幾旬
        self.tt = random.choice([0,6,9,15,27]) if self.ifRandtt else 0      #隨機初始起始月份
        self.S = 400    # 初始水庫水量(萬噸)
        
        scenerio = np.random.choice([i for i in range(len(self.rivers))], p=self.river_p)
        self.river = self.rivers[scenerio]         #隨機選擇入流情境
        self.inflow = self.river[self.t+self.tt]

        d_scenerio = np.random.choice([j for j in range(len(self.demands))], p=self.demands_p)
        self.dmd = self.demands[d_scenerio]        #隨機選擇demand情境
        self.demand = self.dmd[self.t+self.tt] + random.randint(-20,20)*10       

        #================================Normalization===================================
        norm_S = 2*(self.S)/self.max_storage -1
        #norm_Zmin =  2*(0 - (-100))/(0-(-100))-1
        norm_demand = 2*(self.demand-np.min(self.demands)+200)/(np.max(self.demands)-np.min(self.demands)+400) -1
        self.norm_inflow_t_1 = 0
        self.norm_inflow_t_2 = 0
        self.norm_inflow_t_3 = 0
        self.norm_inflow_t_4 = 0

        return np.array([norm_S, 
                         #norm_Zmin, 
                         norm_demand, 
                         self.norm_inflow_t_1, 
                         self.norm_inflow_t_2, 
                         self.norm_inflow_t_3, 
                         self.norm_inflow_t_4 ])
    
 
    def step(self, action):
        p = action[1]*120    # pumping rate(m^3/day)
        GW = p*2151*10/10000   # 2151口井抽一旬的水量(萬噸)
        
        Q = {0: self.pump(-p, self.wel_list)}
        
        #recharge=(self.inflow-np.min(self.rivers))/(np.max(self.rivers)-np.min(self.rivers))*(0.00003-0.000024)+0.000024
        #rech={0: 0.00005, 1: recharge}
        
        #===============================MODFLOW_2005===================================
        if self.t != 0:
            bas = flopy.modflow.ModflowBas(self.mf, ibound=self.ibound, strt=self.head, hnoflo=999.)
        
        wel = flopy.modflow.ModflowWel(self.mf, stress_period_data=Q)
        #rch = flopy.modflow.ModflowRch(self.mf, rech=rech)
        
        # Fix DIS file 
        with open('choshui.dis', 'r') as f:
            lines = f.readlines()
        new_last_line = f'     {self.day[(self.t+self.tt)%36]:.6f}             1  1.000000  TR '
        lines[-1] = new_last_line
        with open('choshui.dis', 'w') as f:
            f.writelines(lines)
        
        # Write the model input files
        self.mf.write_input()   
        success, buff = self.mf.run_model(silent=True)    # Run the model
        if not success:
            raise Exception("MODFLOW did not terminate normally.")
        
        hds = bf.HeadFile(self.modelname+'.hds')
        times = hds.get_times()
        self.head = hds.get_data(totim=times[-1])
        
        #================================Compute Reward=================================
        SW = action[0]*16*86400*10/10000     # Quantity of SW intake: 0 ~ 160cms x 10days
        if SW > self.inflow*864000/10000 + self.S:
            SW = self.inflow*864000/10000 + self.S
            self.S = 0
            reward = -2
        else:
            self.S = min(self.S - SW + self.inflow*864000/10000, self.max_storage)   #水庫平衡
            
            WDR = (self.demand - SW - GW)/self.demand
            reward = 1-3*abs(WDR)
            if (self.demand - SW - GW) >=-300 and (self.demand - SW - GW)<=0:
                reward += 1
           
        self.sw = SW     # For checking SW value
        
        # Bonus for SW saving when wet period 
        if self.S/self.max_storage >= 0.9 and action[1]==0:
            reward +=1 
        
        # Constraints of groundwater pumping
        self.obs_wels_h_now = []
        penalty = 0
        for i, (o, xy) in enumerate(sorted(self.obs_wels.items())):
            h = self.head[0][399-xy[0]][xy[1]]
            self.obs_wels_h_now.append(h)
            if self.obs_wels_h_former[i] - h > 0.5:
                penalty += 1
        
        reward += -penalty 

        #=================================transition====================================
        done = False     
        if self.t==self.t_final-1 :
            done = True
        else:
            self.t += 1
        
        self.obs_wels_h_former = self.obs_wels_h_now
        
        self.norm_inflow_t_4 = self.norm_inflow_t_3
        self.norm_inflow_t_3 = self.norm_inflow_t_2
        self.norm_inflow_t_2 = self.norm_inflow_t_1
        self.norm_inflow_t_1 = 2*(self.inflow-np.min(self.rivers))/(np.max(self.rivers)-np.min(self.rivers)) -1
        #self.month=((self.t+self.tt)//3)%12+1
        
        #每跑完一年(self.t %36==0)，隨機更換情境年
        scenerio = np.random.choice([i for i in range(len(self.rivers))], p=self.river_p)
        self.river = self.rivers[scenerio] if self.t %36==0 else self.river
        d_scenerio = np.random.choice([j for j in range(len(self.demands))], p=self.demands_p)
        self.dmd = self.demands[d_scenerio] if self.t %36==0 else self.dmd 
        
        self.demand = self.dmd[(self.t+self.tt)%36]+random.randint(-20,20)*10     
        self.inflow = self.river[(self.t+self.tt)%36]

        #state各項目標準化至[-1,1]
        norm_S = 2*(self.S)/self.max_storage -1
        #norm_Zmin =  2*(Z.min() - (-100))/(0-(-100))-1
        #norm_Zmin = 0
        #norm_month = 2*(self.month-1)/(12-1) -1
        norm_demand = 2*(self.demand-np.min(self.demands)+200)/(np.max(self.demands)-np.min(self.demands)+400) -1

        return np.array([norm_S, 
                         #norm_Zmin, 
                         norm_demand, 
                         self.norm_inflow_t_1, 
                         self.norm_inflow_t_2, 
                         self.norm_inflow_t_3, 
                         self.norm_inflow_t_4 ]), reward, done, {}
                         